Nama: Hanun Masitha Ramadhani

In [1]:
import pandas as pd
df = pd.read_csv('/content/song.csv')
df = df[:20000]

In [2]:
song_dict1 = {}
song_dict2 = {}
for x, y in zip(df['song_id'], df['title']):
    if x not in song_dict1.keys():
        song_dict1[x] = y
        song_dict2[y] = x

(1) Buatlah sistem rekomendasi popularity-based berupa fungsi kustom yang menerima 3 argumen: dataset, jumlah lagu yang direkomendasikan, dan minimal jumlah rating lagu. Print rekomendasi top 10 lagu.

In [3]:
popular = pd.DataFrame({'count':df.groupby('song_id').count()['user_id'],
                       'rating':df.groupby('song_id').mean()['rating']})
popular

,count,rating
song_id,,
16,667,1.161919
45,113,1.035398
108,143,1.104895
267,269,1.059480
315,104,1.153846
...,...,...
9562,184,1.228261
9614,84,1.083333
9775,104,1.096154


In [4]:
popular['rating'].max()

1.6904761904761905

In [5]:
popular['rating'].min()

1.0149253731343284

In [6]:
def popular_recom(dataset, min_rating, k):
    a = []
    popular2 = dataset[dataset['rating']>=min_rating].sort_values('rating', ascending=False)
    for x in popular2.index[:k]:
        a.append(song_dict1[x])
    return a

In [7]:
popular_recom(popular, 1.5, 10)

['Disconnect_ Disconnect',
 'OVO MI JE `KOLA',
 'Here I Am To Worship (Here I Am To Worship Album Version)',
 'City Of Angels',
 'The Stallion']

(2) Buatlah sistem rekomendasi collaborative filtering user-user. Hitung RMSE dari sistem rekomendasi yang dibuat. Buat sistem rekomendasi berupa fungsi kustom yang menerima 4 argumen: dataset, user id, jumlah lagu yang direkomendasikan, dan algoritma. Pastikan rekomendasi yang diberikan adalah lagu yang belum diberi rating oleh user. Print rekomendasi top 10 lagu untuk salah satu user.

In [8]:
!pip install scikit-surprise
from surprise import KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

In [9]:
df2 = df[['user_id', 'song_id', 'rating']]

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  20000 non-null  int64  
 1   song_id  20000 non-null  int64  
 2   rating   20000 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 468.9 KB


In [11]:
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(df2, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [12]:
data

In [15]:
param_grid = {'k': [30, 40],
             'min_k': [6, 9],
             'sim_options' : {'name': ['msd', 'pearson'],
                             'user_based': [True]}}
gscv = GridSearchCV(KNNBasic, param_grid, cv=4, measures = ['rmse'])

In [16]:
gscv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarit

In [17]:
gscv.best_score

{'rmse': 0.47045993909836636}

In [18]:
gscv.best_params

{'rmse': {'k': 30,
  'min_k': 9,
  'sim_options': {'name': 'msd', 'user_based': True}}}

In [19]:
knn_user = KNNBasic(**gscv.best_params)
knn_user.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [37]:
def get_recom_user(dataset, user, k, algo):
    dataset_user = pd.pivot_table(dataset, index='user_id', columns='song_id').loc[user]
    dataset_user_null = dataset_user[dataset_user.isnull()].reset_index()
    non_interacted = []
    for x,y in zip(dataset_user_null.loc[:,user], dataset_user_null.loc[:,'song_id']):
        if pd.isnull(x) == True:
            non_interacted.append(y)
    non_interacted_df = {'movie':{}, 'estimated_rating':{}}
    for x, y in zip(non_interacted, range(len(non_interacted))):
        non_interacted_df['movie'][y] = song_dict1[x]
        non_interacted_df['estimated_rating'][y] = algo.predict(user, x).est
    f = pd.DataFrame(non_interacted_df).sort_values('estimated_rating', ascending=False).iloc[:k,:]
    return f

In [38]:
get_recom_user(df2, 66345, 10, knn_user)

,movie,estimated_rating
39,Gel (LP Version),2.250000
84,Love Is Here,2.000000
63,Habana,2.000000
22,Sing_ Sing_ Sing (Key-E-Premiere Performance P...,2.000000
38,Hell On The Heart,2.000000
86,Bar Tacuba,2.000000
53,D.O.A. Part 2,1.782609
87,Hymne A L'Amour (Album Version),1.666667
28,Amazing Grace (1999 Digital Remaster),1.500000
25,OVO MI JE `KOLA,1.500000


In [33]:
b = []
c = []
for x in df2['song_id'].unique():
    if x in df2[df2['user_id']==66345]['song_id'].unique():
        b.append(x)
    else:
        c.append(x)

In [34]:
b

[5076, 8178, 6095, 7562, 1050, 1998]

In [35]:
c

[2709,
 7147,
 320,
 3398,
 9384,
 593,
 2792,
 1199,
 7406,
 874,
 1310,
 2212,
 8867,
 9534,
 2471,
 4160,
 6348,
 16,
 7266,
 980,
 6840,
 9173,
 3525,
 804,
 6428,
 9064,
 4095,
 4464,
 8127,
 1519,
 1733,
 5266,
 7259,
 3583,
 8091,
 8862,
 9558,
 9614,
 108,
 2706,
 6882,
 950,
 6761,
 8939,
 378,
 6249,
 6689,
 560,
 9921,
 8637,
 4085,
 2972,
 2118,
 6728,
 3598,
 2768,
 5619,
 2749,
 7775,
 2055,
 3274,
 267,
 5278,
 6408,
 4289,
 3483,
 4689,
 2934,
 2241,
 5486,
 7662,
 5518,
 9936,
 3080,
 4477,
 7228,
 2464,
 8259,
 8164,
 859,
 8927,
 9775,
 45,
 8392,
 1540,
 7794,
 8648,
 7012,
 3726,
 3727,
 6077,
 776,
 7922,
 9562,
 4063,
 3110,
 8518,
 6111,
 6186,
 8999,
 8723,
 315,
 5959,
 9348]

(3) Buatlah sistem rekomendasi collaborative filtering item-item. Hitung RMSE dari sistem rekomendasi yang dibuat. Buat sistem rekomendasi berupa fungsi kustom yang menerima 4 argumen: dataset, user id, jumlah lagu yang direkomendasikan, dan algoritma. Pastikan rekomendasi yang diberikan adalah lagu yang belum diberi rating oleh user. Print rekomendasi top 10 lagu untuk salah satu user.

In [39]:
param_grid = {'k': [30, 40],
             'min_k': [6, 9],
             'sim_options' : {'name': ['msd', 'pearson'],
                             'item_based': [True]}}
gscv = GridSearchCV(KNNBasic, param_grid, cv=4, measures = ['rmse'])

In [40]:
gscv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarit

In [41]:
gscv.best_score

{'rmse': 0.4708624448939511}

In [42]:
gscv.best_params

{'rmse': {'k': 30,
  'min_k': 9,
  'sim_options': {'item_based': True, 'name': 'msd', 'user_based': True}}}

In [43]:
knn_item = KNNBasic(**gscv.best_params)
knn_item.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [44]:
get_recom_user(df2, 66345, 10, knn_item)

,movie,estimated_rating
39,Gel (LP Version),2.250000
84,Love Is Here,2.000000
63,Habana,2.000000
22,Sing_ Sing_ Sing (Key-E-Premiere Performance P...,2.000000
38,Hell On The Heart,2.000000
86,Bar Tacuba,2.000000
53,D.O.A. Part 2,1.782609
87,Hymne A L'Amour (Album Version),1.666667
28,Amazing Grace (1999 Digital Remaster),1.500000
25,OVO MI JE `KOLA,1.500000
